In [1]:
import torch
import gymnasium as gym
import numpy as np
from transformers import DecisionTransformerModel
import gymnasium as gym
import os
import pandas as pd
from finrl.main import check_and_make_directories
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from stable_baselines3 import PPO
from finrl.agents.stablebaselines3.models import DRLAgent


check_and_make_directories([TRAINED_MODEL_DIR])

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
device = torch.device('cpu')
torch.cuda.current_device()

0

In [12]:
train = pd.read_csv('data/train.csv')
train = train.set_index(train.columns[0])
train.index.names =['']

trade = pd.read_csv('data/trade.csv')
trade = trade.set_index(trade.columns[0])
trade.index.names = ['']

In [4]:
# Load train and trade DataFrames from pickle files instead of CSV
train = pd.read_pickle('data/train.pickle')
train.index.names = ['']

trade = pd.read_pickle('data/trade.pickle')
trade.index.names = ['']


In [5]:
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 394, State Space: 3941


In [6]:
buy_cost_list = sell_cost_list = [0.005] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

e_trade_gym = StockTradingEnv(df=trade, **env_kwargs)
env, obs = e_trade_gym.get_sb_env()

Here `scale=1000.0` is the normalization factor used training to scale return-to-go. The same value used in the data collator (`DecisionTransformerGymCollator`). It also ensures consistent scaling between training and evalution.

In [7]:
scale = 1000.0

Target performance goal for the Decision Transformer (3.6 normalized units). Represents a desired cumulative return of 3600, and the DT will condition its action on achieving this target return. This is the key advantage of Decision Transformers - you can specify different performance goals without retraining.

In [8]:
TARGET_RETURN = 3600 / scale

We have the state space dimension, extract the observation space size from the trading environment, and ensure compatbility between the model and environment.

In [9]:
state_dim = env.observation_space.shape[0]

Here we have the action space dimension, which extracts the action space size from the trading enviroment. This matches the number of stocks being traded.

In [10]:
act_dim = env.action_space.shape[0]

In [11]:
state_dim, act_dim

(3941, 394)

In [12]:
from datasets import load_from_disk

dataset = load_from_disk('data/dataset/')

state_mean = dataset['state_mean']
state_std = dataset['state_std']

state_mean = state_mean[:state_dim]
state_std = state_std[:state_dim]

state_mean = torch.Tensor(state_mean).to(device)
state_std = torch.Tensor(state_std).to(device)

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

In [13]:
# Create the decision transformer model
model = DecisionTransformerModel.from_pretrained('trained_models')
model = model.to(device)

print(list(model.encoder.parameters()))

[Parameter containing:
tensor([[ 0.0128, -0.0341, -0.0044,  0.0166,  0.0122,  0.0242, -0.0028, -0.0047,
          0.0233, -0.0024, -0.0091,  0.0192, -0.0156, -0.0004, -0.0162, -0.0014,
         -0.0035, -0.0020, -0.0028,  0.0152,  0.0119,  0.0075, -0.0005, -0.0363,
          0.0218, -0.0098,  0.0107, -0.0255, -0.0226,  0.0035,  0.0130, -0.0247,
         -0.0025,  0.0002, -0.0202,  0.0056, -0.0101,  0.0394, -0.0184,  0.0301,
          0.0017, -0.0112, -0.0177, -0.0022, -0.0097,  0.0270,  0.0065,  0.0073,
         -0.0181, -0.0062, -0.0447,  0.0112, -0.0008, -0.0002,  0.0188, -0.0085,
         -0.0292,  0.0458, -0.0058,  0.0001, -0.0148,  0.0200, -0.0132,  0.0167,
         -0.0113,  0.0187,  0.0211,  0.0154,  0.0254, -0.0035, -0.0044,  0.0016,
         -0.0069,  0.0028, -0.0158,  0.0141, -0.0096,  0.0300, -0.0063,  0.0408,
         -0.0039, -0.0118, -0.0465,  0.0059,  0.0158, -0.0405, -0.0187, -0.0087,
         -0.0003, -0.0008,  0.0157, -0.0109,  0.0011, -0.0051, -0.0198, -0.0023,
     

### Prediction


Here we are defining the `get_action` function, where is the core inference function for the Decision Transformer during evaluation. It performs autoregression action prediction using a sliding window approach.

This function takes the current trading history and predicts the next action the Decision Transformer should take, conditioned on the desired return-to-go target.

The autoregressive generation uses the previous 20 timesteps to predict the next action. Each prediction is based on recent history and target return.

Actions are conditioned on the target performance and the model learns to achieve the specified return target.

#### Input Reshaping (Line 5-8)

* Reshape all inputs to batch format: `(batch_size=1, sequences_length, feature_dim)`
* Prepares data for the transformer model

#### Sliding Window (Lines 12-16)

* Takes only the last `max_length` timesteps (typically 20)
* Creates a sliding window of recent history
* Calcualtes how much padding is needed

#### Padding (Lines 19-24)

* Pads sequences to a fixed length
* Adds zeros at the beginning for padding
* Creates attention mask: 0 for padding, 1 for real data

#### Model Prediction (Lines 26-32)

* Feeds the padded sequence to the Decision Transformer
* Model processes the sequence through its attention layers
* Returns predictions for states, actions, return-to-go

#### Extract Action (Lines 34)
* Returns only the last predicted action `([0,1])`
* This is the action for the current timestep
* Ignores predictions for previous timesteps

In [14]:
# Function that gets an action from the model using autoregressive prediction with a window of the previous 20 timesteps.
def get_action(model, states, actions, rewards, returns_to_go, timesteps):
    # This implementation does not condition on past rewards

    # Reshape inputs to batch format
    states = states.reshape(1, -1, model.config.state_dim)
    actions = actions.reshape(1, -1, model.config.act_dim)
    returns_to_go = returns_to_go.reshape(1, -1, 1)
    timesteps = timesteps.reshape(1, -1)

    #Sliding Window 
    states = states[:, -model.config.max_length :]
    actions = actions[:, -model.config.max_length :]
    returns_to_go = returns_to_go[:, -model.config.max_length :]
    timesteps = timesteps[:, -model.config.max_length :]
    padding = model.config.max_length - states.shape[1]
    
    # pad all tokens to sequence length
    attention_mask = torch.cat([torch.zeros(padding), torch.ones(states.shape[1])]).to(device=device)
    attention_mask = attention_mask.to(dtype=torch.long).reshape(1, -1)
    states = torch.cat([torch.zeros((1, padding, model.config.state_dim)), states], dim=1).float().to(device=device)
    actions = torch.cat([torch.zeros((1, padding, model.config.act_dim)), actions], dim=1).float().to(device=device)
    returns_to_go = torch.cat([torch.zeros((1, padding, 1)), returns_to_go], dim=1).float().to(device=device)
    timesteps = torch.cat([torch.zeros((1, padding), dtype=torch.long), timesteps], dim=1).to(device=device)

    state_preds, action_preds, return_preds = model(
        states=states,
        actions=actions,
        rewards=rewards,
        returns_to_go=returns_to_go,
        timesteps=timesteps,
        attention_mask=attention_mask,
        return_dict=False,
    )

    return action_preds[0, -1]

In [15]:
max_steps = e_trade_gym.df.tic.count() - 1
max_steps

np.int64(180845)

### Evaluation

This is the main evluation loop for the Decision Transformer, where it interacts with the trading environment to test the model's performance.

#### Initialization (Lines 4-17)

* Resets environment and initializes tracking variables
* Sets targets return (the performance goal the DT should achieve)
* Initializes empty tensors for states, actions, rewards, and timesteps
* Prepares data structures for the autoregressive generation

#### Action Prediction (Lines 20-32)

* Adds placeholder for current action and reward
* Calls the Decision Transformer to predict next trading action
* uses normalized states (subtracts mean, divides by std)

#### Environment Interaction (Lines 35-40)
* Executes the predicted action in the trading environment
* Receives new state and reward from the environment
* Updates the state and reward history

#### Target Return Updates (Lines 42-44)
* Updates the target return by subtracting the achieved reward
* Maintains the remaining target for future timesteps
* Updates timesteps counter

In [16]:
# Interact with the environment
#max_ep_len = 1000

episode_return, episode_length = 0, 0
state = env.reset()
target_return = torch.tensor(TARGET_RETURN, device=device, dtype=torch.float32).reshape(1, 1)
states = torch.from_numpy(state).reshape(1, state_dim).to(device=device, dtype=torch.float32)
actions = torch.zeros((0, act_dim), device=device, dtype=torch.float32)
rewards = torch.zeros(0, device=device, dtype=torch.float32)

timesteps = torch.tensor(0, device=device, dtype=torch.long).reshape(1, 1)

#max_steps = e_trade_gym.df.tic.count() - 1
max_steps = len(e_trade_gym.df.index.unique()) - 1

account_memory = None  # This help avoid unnecessary list creation
actions_memory = None  # optimize memory consumption

for t in range(max_steps+1):
    actions = torch.cat([actions, torch.zeros((1, act_dim), device=device)], dim=0)
    rewards = torch.cat([rewards, torch.zeros(1, device=device)])

    action = get_action(
        model,
        (states - state_mean) / state_std,
        actions,
        rewards,
        target_return,
        timesteps,
    )

    actions[-1] = action
    actions_numpy = actions.detach().cpu().numpy()

    #state, reward, done, truncated, info = env.step(actions_numpy)
    state, reward, done, _ = env.step(actions_numpy)

    cur_state = torch.from_numpy(state).to(device=device).reshape(1, state_dim)
    states = torch.cat([states, cur_state], dim=0)
    rewards[-1] = torch.from_numpy(reward).to(device=device)

    pred_return = target_return[0, -1] - (reward / scale)
    target_return = torch.cat([target_return, pred_return.reshape(1, 1)], dim=1)
    timesteps = torch.cat([timesteps, torch.ones((1, 1), device=device, dtype=torch.long) * (t + 1)], dim=1)

    episode_return += reward
    episode_length += 1

    if (t == max_steps - 1):  # more descriptive condition for early termination to clarify the logic
        # Call instance methods of vectorized environments
        # https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html
        account_memory = env.env_method(method_name="save_asset_memory")
        actions_memory = env.env_method(method_name="save_action_memory")

    if done[0]:# or truncated:
        break

C:\Users\drebi\AppData\Local\Temp\ipykernel_38216\392077640.py:42: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  pred_return = target_return[0, -1] - (reward / scale)
C:\Users\drebi\AppData\Local\Temp\ipykernel_38216\392077640.py:42: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  pred_return = target_return[0, -1] - (reward / scale)


In [17]:
df_account_value_dt, df_actions_dt = account_memory[0], actions_memory[0]

In [18]:
df_account_value_dt.shape, df_actions_dt.shape

((459, 2), (458, 394))

In [19]:
df_account_value_dt.head()

,date,account_value
0,2022-07-01,1.000000e+06
1,2022-07-05,9.986497e+05
2,2022-07-06,9.992926e+05
3,2022-07-07,1.004898e+06
4,2022-07-08,1.002063e+06


In [20]:
df_account_value_dt.tail()

,date,account_value
454,2024-04-23,1.285404e+06
455,2024-04-24,1.285611e+06
456,2024-04-25,1.281273e+06
457,2024-04-26,1.283513e+06
458,2024-04-29,1.288143e+06


In [21]:
df_actions_dt.head()

,A,AAL,AAP,AAPL,ABT,ACN,ADBE,ADI,ADM,ADP,...,WU,WY,WYNN,XEL,XOM,XRAY,XRX,YUM,ZBH,ZION
date,,,,,,,,,,,,,,,,,,,,,
2022-07-01,13,3,6,0,0,0,0,5,0,0,...,0,0,0,0,0,20,5,11,5,3
2022-07-05,13,3,6,0,0,0,0,5,0,0,...,0,0,0,0,0,20,5,11,5,3
2022-07-06,13,3,6,0,0,0,0,5,0,0,...,0,0,0,0,0,20,5,11,5,3
2022-07-07,13,3,6,0,0,0,0,5,0,0,...,0,0,0,0,0,20,5,11,5,3
2022-07-08,13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,20,0,11,0,0


In [22]:
df_actions_dt.tail()

,A,AAL,AAP,AAPL,ABT,ACN,ADBE,ADI,ADM,ADP,...,WU,WY,WYNN,XEL,XOM,XRAY,XRX,YUM,ZBH,ZION
date,,,,,,,,,,,,,,,,,,,,,
2024-04-22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2024-04-23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2024-04-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2024-04-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2024-04-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df_account_value_dt.to_pickle('data/df_account_value_dt.pkl')
df_actions_dt.to_pickle('data/df_actions_dt.pkl')

In [24]:
trained_PPO = PPO.load('trained_models/agent_ppo.zip')

c:\Users\drebi\miniconda3\envs\summerresearch\Lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [25]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_PPO,
    environment=e_trade_gym
)

hit end!


In [26]:
df_account_value_ppo.to_pickle('data/df_account_value_ppo.pkl')
df_actions_ppo.to_pickle('data/df_actions_ppo.pkl')